In [87]:
import numpy as np
import pandas as pd

from __future__ import print_function
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [88]:
# Read Dataset
df = pd.read_csv('kdd-binary-dataset.csv')
df.head()

,Duration,Protocol_Type,Service,Flag,Src_Bytes,Dst_Bytes,Land,Wrong_Fragment,Urgent,Hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack
0,0,tcp,private,REJ,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly
1,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,anomaly
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,anomaly


**Coverting Categorical from String into Numeric Data.**

Protocol_Type
tcp: 1, udp: 2, icmp: 3

Service
private: 1, ftp_data: 2, eco_i: 3, telnet: 4, etc...

Flag
REJ: 1, SF: 2, RSTO: 3, etc...

In [89]:
# Protocol Type Conversion
df["Protocol_Type"]=df["Protocol_Type"].replace({"tcp":1})
df["Protocol_Type"]=df["Protocol_Type"].replace({"udp":2})
df["Protocol_Type"]=df["Protocol_Type"].replace({"icmp":3})

In [90]:
# Creating new columns for numeric codes of Service and Flag
df['Serv_code'] = pd.factorize(df['Service'])[0] + 1
df['Flag_code'] = pd.factorize(df['Flag'])[0] + 1

In [91]:
df.head()

,Duration,Protocol_Type,Service,Flag,Src_Bytes,Dst_Bytes,Land,Wrong_Fragment,Urgent,Hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack,Serv_code,Flag_code
0,0,1,private,REJ,0,0,0,0,0,0,...,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly,1,1
1,0,1,private,REJ,0,0,0,0,0,0,...,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly,1,1
2,2,1,ftp_data,SF,12983,0,0,0,0,0,...,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,2,2
3,0,3,eco_i,SF,20,0,0,0,0,0,...,0.00,1.00,0.28,0.0,0.0,0.00,0.00,anomaly,3,2
4,1,1,telnet,RSTO,0,15,0,0,0,0,...,0.17,0.03,0.02,0.0,0.0,0.83,0.71,anomaly,4,3


**Converting Attack Types into Binary Classification 0 and 1.**

*   Anomaly: 1
*   Normal: 0


cols = ['Attack', 'Duration', 'Protocol_Type', 'Src_Bytes','Dst_Bytes', 'Land', 'Wrong_Fragment', 'Urgent', 'Hot', 'Num_Failed_Logins', 'Logged_in', 'Num_Compromised', 'Root_shell', 'Su_attempted', 'Num_Root', 'Num_File_Creations', 'Num_shells', 'Num_access_files', 'Num_Outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Serv_code', 'Flag_code']

In [92]:
df["Attack"]=df["Attack"].replace({"anomaly":1})
df["Attack"]= df["Attack"].replace({"normal":0})

In [93]:
df.head()

,Duration,Protocol_Type,Service,Flag,Src_Bytes,Dst_Bytes,Land,Wrong_Fragment,Urgent,Hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack,Serv_code,Flag_code
0,0,1,private,REJ,0,0,0,0,0,0,...,0.06,0.00,0.00,0.0,0.0,1.00,1.00,1,1,1
1,0,1,private,REJ,0,0,0,0,0,0,...,0.06,0.00,0.00,0.0,0.0,1.00,1.00,1,1,1
2,2,1,ftp_data,SF,12983,0,0,0,0,0,...,0.04,0.61,0.02,0.0,0.0,0.00,0.00,0,2,2
3,0,3,eco_i,SF,20,0,0,0,0,0,...,0.00,1.00,0.28,0.0,0.0,0.00,0.00,1,3,2
4,1,1,telnet,RSTO,0,15,0,0,0,0,...,0.17,0.03,0.02,0.0,0.0,0.83,0.71,1,4,3


**Normalizing the Features of our Training Dataset**

In [94]:
x = df.drop(["Attack", "Service", "Flag"], axis=1)
y= df["Attack"]
normalized_arr = preprocessing.normalize(x)
print(normalized_arr)

[[0.00000000e+00 2.91515444e-03 0.00000000e+00 ... 2.91515444e-03
  2.91515444e-03 2.91515444e-03]
 [0.00000000e+00 3.46002104e-03 0.00000000e+00 ... 3.46002104e-03
  3.46002104e-03 3.46002104e-03]
 [1.54036010e-04 7.70180048e-05 9.99924756e-01 ... 0.00000000e+00
  1.54036010e-04 1.54036010e-04]
 ...
 [0.00000000e+00 1.81253895e-05 9.88558745e-01 ... 1.26877727e-06
  9.06269477e-05 3.62507791e-05]
 [0.00000000e+00 5.49826046e-03 1.15463470e-01 ... 0.00000000e+00
  3.84878232e-02 5.49826046e-03]
 [0.00000000e+00 3.88469598e-03 0.00000000e+00 ... 3.88469598e-03
  1.01002095e-01 3.88469598e-03]]


In [95]:
norm_df=pd.DataFrame(normalized_arr)
norm_df.columns = x.columns
norm_df["Attack"]=y
norm_df.head()

,Duration,Protocol_Type,Src_Bytes,Dst_Bytes,Land,Wrong_Fragment,Urgent,Hot,Num_Failed_Logins,Logged_in,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Serv_code,Flag_code,Attack
0,0.000000,0.002915,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000175,0.000000,0.000000,0.0,0.0,0.002915,0.002915,0.002915,0.002915,1
1,0.000000,0.003460,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000208,0.000000,0.000000,0.0,0.0,0.003460,0.003460,0.003460,0.003460,1
2,0.000154,0.000077,0.999925,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000003,0.000047,0.000002,0.0,0.0,0.000000,0.000000,0.000154,0.000154,0
3,0.000000,0.033731,0.224874,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.011244,0.003148,0.0,0.0,0.000000,0.000000,0.033731,0.022487,1
4,0.010810,0.010810,0.000000,0.162144,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001838,0.000324,0.000216,0.0,0.0,0.008972,0.007675,0.043238,0.032429,1


**Splitting the Dataset into Training and Testing Dataset**

Here, we have considered the ratio of 60:40 to split the dataset into train and test dataset respectively.

We can clearly observe that the dataset is splitted into train and test data with a ratio of 60:40

In [96]:
X_train, X_test, y_train, y_test = train_test_split(x,y,train_size=0.6,test_size=0.4, random_state=42)

**Feature Selection**

In [97]:
# Feature Selection
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [98]:
sel.get_support()

array([ True,  True,  True,  True, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True, False, False,
       False,  True,  True,  True, False])

In [99]:
# Feature Selected
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

13

In [100]:
print(selected_feat)

Index(['Duration', 'Protocol_Type', 'Src_Bytes', 'Dst_Bytes', 'Logged_in',
       'count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Serv_code'],
      dtype='object')


In [101]:
# X_train & X_test data
X_train = X_train[['Duration', 'Protocol_Type', 'Src_Bytes', 'Dst_Bytes', 'Logged_in',
       'count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Serv_code']]
X_test = X_test[['Duration', 'Protocol_Type', 'Src_Bytes', 'Dst_Bytes', 'Logged_in',
       'count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Serv_code']]

In [102]:
X_test = X_test.to_numpy()
X_train = X_train.to_numpy()

In [103]:
# Shape X_train data
X_train.shape

(13526, 13)

In [104]:
# Batch size
batch_size = 32

In [105]:
# Reshape input to be [samples, time steps, features]
X_train  = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [106]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (13526, 1, 13)
y_train shape: (13526,)
X_test shape: (9018, 1, 13)
y_test shape: (9018,)


In [107]:
# Model
model = Sequential()
model.add(LSTM(4,input_dim=13))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
print(model.get_config())

{'name': 'sequential_5', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, None, 13), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_5_input'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'LSTM', 'config': {'name': 'lstm_5', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, None, 13), 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 4, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'recurrent_initializer': {'module': 'keras.initializers', 'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'r

In [108]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 4)                 288       
                                                                 
 dropout_5 (Dropout)         (None, 4)                 0         
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
 activation_5 (Activation)   (None, 1)                 0         
                                                                 
Total params: 293 (1.14 KB)
Trainable params: 293 (1.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [111]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.save("lstm1layer_model.hdf5")

Epoch 1/10
408/423 [===========================>..] - ETA: 0s - loss: 0.4021 - accuracy: 0.8563

423/423 [==============================] - 4s 5ms/step - loss: 0.4026 - accuracy: 0.8559 - val_loss: 0.3509 - val_accuracy: 0.9004
Epoch 2/10
404/423 [===========================>..] - ETA: 0s - loss: 0.3998 - accuracy: 0.8589

423/423 [==============================] - 2s 4ms/step - loss: 0.3993 - accuracy: 0.8594 - val_loss: 0.3529 - val_accuracy: 0.9004
Epoch 3/10
411/423 [============================>.] - ETA: 0s - loss: 0.4012 - accuracy: 0.8561

423/423 [==============================] - 2s 5ms/step - loss: 0.4011 - accuracy: 0.8566 - val_loss: 0.3459 - val_accuracy: 0.9031
Epoch 4/10
416/423 [============================>.] - ETA: 0s - loss: 0.4036 - accuracy: 0.8527

423/423 [==============================] - 2s 5ms/step - loss: 0.4026 - accuracy: 0.8533 - val_loss: 0.3480 - val_accuracy: 0.8971
Epoch 5/10
411/423 [============================>.] - ETA: 0s - loss: 0.3981 - accuracy: 0.8559

...

423/423 [==============================] - 1s 3ms/step - loss: 0.3962 - accuracy: 0.8578 - val_loss: 0.3396 - val_accuracy: 0.9013
Epoch 7/10
417/423 [============================>.] - ETA: 0s - loss: 0.3911 - accuracy: 0.8582

423/423 [==============================] - 2s 4ms/step - loss: 0.3911 - accuracy: 0.8583 - val_loss: 0.3420 - val_accuracy: 0.8994
Epoch 8/10
405/423 [===========================>..] - ETA: 0s - loss: 0.3991 - accuracy: 0.8555

423/423 [==============================] - 2s 4ms/step - loss: 0.3975 - accuracy: 0.8566 - val_loss: 0.3402 - val_accuracy: 0.9009
Epoch 9/10
419/423 [============================>.] - ETA: 0s - loss: 0.3965 - accuracy: 0.8572

423/423 [==============================] - 2s 4ms/step - loss: 0.3961 - accuracy: 0.8573 - val_loss: 0.3350 - val_accuracy: 0.9011
Epoch 10/10
422/423 [============================>.] - ETA: 0s - loss: 0.3961 - accuracy: 0.8570

423/423 [==============================] - 2s 4ms/step - loss: 0.3961 - accuracy: 0.8569 - val_loss: 0.3380 - val_accuracy: 0.9001


In [112]:
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict(X_test)
classes_x=np.argmax(y_pred,axis=1)

282/282 [==============================] - 0s 2ms/step - loss: 0.3380 - accuracy: 0.9001

Loss: 0.34, Accuracy: 90.01%
282/282 [==============================] - 1s 2ms/step
